In [0]:
from zipfile import ZipFile
import numpy as np
import pickle
import pandas as pd

In [0]:
! wget --user rijulganguly --password rZQRfkap http://www.eecs.qmul.ac.uk/mmv/datasets/deap/data/data_preprocessed_python.zip

--2019-10-20 18:08:06--  http://www.eecs.qmul.ac.uk/mmv/datasets/deap/data/data_preprocessed_python.zip
Resolving www.eecs.qmul.ac.uk (www.eecs.qmul.ac.uk)... 138.37.95.147
Connecting to www.eecs.qmul.ac.uk (www.eecs.qmul.ac.uk)|138.37.95.147|:80... connected.
HTTP request sent, awaiting response... 502 Proxy Error
2019-10-20 18:09:51 ERROR 502: Proxy Error.



In [0]:
filename = "data_preprocessed_python.zip"
m = "data_preprocessed_python/s"
num = 1

In [0]:
dest_dir = "/content"
with ZipFile(filename) as zf:
    zf.extractall(dest_dir)


     |████████████████████████████████| 778kB 2.8MB/s 
     |████████████████████████████████| 348kB 32.8MB/s 
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 2.0.10 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.8.0 which is incompatible.
     |████████████████████████████████| 122kB 2.8MB/s 
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 2.0.10 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.6.0, but you'll have ipykernel 5.1.3 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.8.0 which is incompatible.


sample_data/


In [0]:
x = pickle.load(open('data_preprocessed_python/s04.dat', 'rb'), encoding='iso-8859-1')

In [0]:
x['labels'][0][0]

6.68

In [0]:
datas = {}

In [0]:
for i in range(1,33):
    if(i<10):
        eID = "0" + str(i)
    else:
        eID = str(i)
        
    fLoad = 'data_preprocessed_python/s' + eID + '.dat'
    dat = pickle.load(open(fLoad, 'rb'), encoding='iso-8859-1')
    
    datas[i] = dat

In [0]:
from scipy.stats import kurtosis
from scipy.stats import skew




In [0]:
changed_data = {}
m_in = np.zeros((40,40,101))

cnt = 0
mj = 0
for i in range(1,33):
    for j in range(40):
        
        for k in range(40):
            mj = 0 
            cnt = 0
            
            for l in range(10):
                m_in[j][k][cnt] = np.mean(datas[i]['data'][j][k][mj:mj+807])
                m_in[j][k][cnt+1] = np.std(datas[i]['data'][j][k][mj:mj+807])
                m_in[j][k][cnt+2] = np.min(datas[i]['data'][j][k][mj:mj+807])
                m_in[j][k][cnt+3] = np.max(datas[i]['data'][j][k][mj:mj+807])
                m_in[j][k][cnt+4] = np.median(datas[i]['data'][j][k][mj:mj+807])
                m_in[j][k][cnt+5] = np.var(datas[i]['data'][j][k][mj:mj+807])
                m_in[j][k][cnt+6] = np.ptp(datas[i]['data'][j][k][mj:mj+807])
                m_in[j][k][cnt+7] = skew(datas[i]['data'][j][k][mj:mj+807])
                m_in[j][k][cnt+8] = kurtosis(datas[i]['data'][j][k][mj:mj+807])
                
                cnt += 9
                mj += 807

                if(mj > 8064):
                    mj = 8064

            m_in[j][k][cnt] = np.mean(datas[i]['data'][j][k][:8064])
            m_in[j][k][cnt+1] = np.std(datas[i]['data'][j][k][:8064])
            m_in[j][k][cnt+2] = np.min(datas[i]['data'][j][k][:8064])
            m_in[j][k][cnt+3] = np.max(datas[i]['data'][j][k][:8064])
            m_in[j][k][cnt+4] = np.median(datas[i]['data'][j][k][:8064])
            m_in[j][k][cnt+5] = np.var(datas[i]['data'][j][k][:8064])
            m_in[j][k][cnt+6] = np.ptp(datas[i]['data'][j][k][:8064])
            m_in[j][k][cnt+7] = skew(datas[i]['data'][j][k][:8064])
            m_in[j][k][cnt+8] = kurtosis(datas[i]['data'][j][k][:8064])

            m_in[j][k][cnt+9] = i

            m_in[j][k][cnt+10] = j+1

    changed_data[i] = m_in
    m_in = np.zeros((40,40,101))



In [0]:
 changed_data[32].transpose().shape

(101, 40, 40)

In [0]:
import torch
import torch.nn as nn
import torch.functional as F


In [0]:
class CNetValence(nn.Module):
    def __init__(self):
        super(CNetValence, self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(1,100, kernel_size=(3,3)),
                                    nn.Tanh()
                                    )
        self.layer2 = nn.Sequential(nn.Conv2d(100,100, kernel_size=(3,3)),
                                    nn.Tanh(),
                                    nn.MaxPool2d(kernel_size=(2,2)),
                                    nn.Dropout(0.25))
        
        self.layer3 = nn.Sequential(nn.Linear(100*18*48, 128),
                                    nn.Tanh(),
                                    nn.Dropout(0.5))
        self.final_layer = nn.Sequential(nn.Linear(128,3),
                                         nn.Softplus())
        
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(40,-1)
        out = self.layer3(out)
        out = self.final_layer(out)
        return out

In [0]:
model = CNetValence()
t = torch.from_numpy(changed_data[32].reshape((40,1,40,101))).float()


In [0]:
otpt = model(t)

In [0]:
otpt.shape

torch.Size([40, 3])

In [0]:
#v,ind = torch.max(otpt,1)

In [0]:
otpt

tensor([[0.5404, 0.7000, 0.6748],
        [0.5468, 0.9656, 0.7143],
        [0.7076, 0.8055, 0.5747],
        [0.4931, 0.9042, 0.6076],
        [0.6904, 1.0092, 0.7034],
        [0.4268, 0.6764, 0.5138],
        [0.5537, 0.8721, 0.6545],
        [0.6370, 0.6938, 0.7884],
        [0.6810, 1.1190, 0.7615],
        [0.4899, 0.8707, 0.6514],
        [0.6138, 0.8304, 0.6692],
        [0.7473, 0.7879, 0.7366],
        [0.6335, 0.6455, 0.7381],
        [0.6162, 0.9297, 0.6868],
        [0.4649, 0.8498, 0.6386],
        [0.5527, 0.8219, 0.7330],
        [0.7362, 0.6538, 0.6068],
        [0.5468, 0.9404, 0.6082],
        [0.6008, 0.7468, 0.8381],
        [0.6858, 0.8552, 0.5984],
        [0.6466, 0.9314, 0.6369],
        [0.5479, 0.6904, 0.5968],
        [0.7546, 0.9109, 0.7021],
        [0.5035, 0.9300, 0.5495],
        [0.5639, 0.7936, 0.8163],
        [0.7024, 0.7934, 0.6789],
        [0.6252, 0.7353, 0.7642],
        [0.6433, 0.7861, 0.6809],
        [0.5669, 0.6564, 0.9103],
        [0.685

In [0]:
changed_labels = {}
m_lab = np.zeros((40,))

for i in range(1,33):
    for j in range(40):

        k = datas[i]['labels'][j][0]
        if(k>6):
            m_lab[j] = 2
        elif(k>4):
            m_lab[j] = 1
        else:
            m_lab[j] = 0

    
    changed_labels[i] = m_lab
    m_lab = np.zeros((40,))


In [0]:
t_lab = torch.from_numpy(changed_labels[32])
t_lab.shape

torch.Size([40])

In [0]:
criterion = nn.CrossEntropyLoss()
#ind_c = oneHot(ind)
#ind_c.shape

In [0]:
#l = criterion(otpt, t_lab.type(torch.LongTensor))
#l

In [0]:
def oneHot(a):
    mt = np.zeros((40,3))

    for i in range(40):
        if(a[i] == 0):
            mt[i][0] = 1
        elif(a[i] == 1):
            mt[i][1] = 1
        else:
            mt[i][2] = 1

    return torch.from_numpy(mt)

In [0]:
import torch.optim as optim
optimizer = optim.SGD(model.parameters(), lr=0.00001, momentum=0.9)
l_arr = []

In [0]:
for epoch in range(250):
    epoch_acc = 0
    num_correct = 0
    for i in range(1,31):
        for j in range(1,31):
            if(j==i):
                continue

            input = torch.from_numpy(changed_data[j].reshape((40,1,40,101))).float()
            labels = torch.from_numpy(changed_labels[j])
            labels.requires_grad=True
            optimizer.zero_grad()

            output = model(input)
            v,ind = torch.max(output,1)
            #ind_n = oneHot(ind)
            #ind_n.requires_grad=True
            
            loss = criterion(output, labels.type(torch.LongTensor))
            #l_arr.append(loss)
            #num_correct += torch.sum(labels.type(torch.LongTensor) == ind)
            loss.backward()
            optimizer.step()

        input = torch.from_numpy(changed_data[i].reshape((40,1,40,101))).float()
        labels = torch.from_numpy(changed_labels[i])
        labels.requires_grad=False

        output = model(input)
        v,ind = torch.max(output,1)

        loss = criterion(output, labels.type(torch.LongTensor))

        num_correct += torch.sum(labels.type(torch.LongTensor) == ind)
        l_arr.append(loss)
        #print("WORKING")



    epoch_acc = num_correct.float()/(30*40)

    if(epoch%10 == 0):
        print("EPOCH ", epoch)
        print("ACCURACY ", epoch_acc)
        print("LOSS", loss)

        

EPOCH  0
ACCURACY  tensor(0.4233)
LOSS tensor(1.0798, grad_fn=<NllLossBackward>)


In [0]:
#class DNNValence(nn.Module):

In [0]:
from matplotlib import pyplot as plt
%matplotlib inline 
plt.plot(l_arr)
plt.show()

EPOCH  1
